In [31]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import re
import wordninja

#NLTK
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from autocorrect import Speller
from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
import plotly.figure_factory as ff

In [3]:
os.chdir('D:/USF/Text Analytics/Employee Satisfaction Meter using Company Reviews')
df=pd.read_excel("Glassdoor Reviews by Company-Formatted.xlsx")
df['Review_Year'] = df.apply(lambda x:x['ReviewDate'].year,axis=1)

In [52]:
df.head()

,CompanyName,AuthorEmploymentStatus,AuthorJobRole,AuthorLocation,Pros,Cons,ReviewDate,ReviewMainText,ReviewTitle,OverallRating,Review_Year,Modified_Cons_Review,Modified_Pros_Review
0,Amazon,Current Employee,QA Analyst,Bangalore,You will love to work with the company.\r\r\n\...,Till my 3.5 years of experience i do not feel ...,2020-03-25,I worked at Amazon full-time for more than 3 y...,"""NA""",4,2020,Till my year of experience i do not feel anyth...,You will love to work with the company Work li...
1,Amazon,Former Employee,Customer Service Specialist,Pune,Great working atmosphere &amp; good management,Not near my house as I had to relocate.,2020-03-25,I worked at Amazon full-time,"""Amazon is Amazing.""",5,2020,Not near my house a I had to relocate,Great working atmosphere amp good management
2,Amazon,Current Employee,Virtual Customer Service Associate,Pune,Work from home\r\r\nDecent pay\r\r\nFlexible T...,Monotonous and repetitive nature\r\r\nSlow car...,2020-03-24,I have been working at Amazon full-time for le...,"""Good Opportunity""",5,2020,Monotonous and repetitive nature Slow career g...,Work from home Decent pay Flexible Timings
3,Amazon,Current Employee,Virtual Customer Service Associate,Pune,"There is a good scope to learn, work can be so...",Monotonous and repetitive nature\r\r\nSlow car...,2020-03-24,I have been working at Amazon full-time for le...,"""More scope to learn""",4,2020,Monotonous and repetitive nature Slow career g...,There is a good scope to learn work can be som...
4,Amazon,Former Employee,Fulfillment Associate,"Rialto, CA",Nothing is good at that place,Everything Is wrong with that place,2020-03-24,I worked at Amazon full-time,"""Trash""",1,2020,Everything Is wrong with that place,Nothing is good at that place


In [33]:
#Function to calculate vocabulary count,top 5 most frequent words used by presidents & sentiment score of the speech
analyzer = SentimentIntensityAnalyzer()
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
spell = Speller(lang='en')
sentiment_score=0.0
def text_preprocess(review_text):
    #Removing all the special characters from the speech text
    for char in '!#$%&@?,.:;+-*/=<>"\'()[\\]X{|}~\n\t':
        review_text = str(review_text).replace(char, ' ')
        #Converting all the words in speech into lower case
    #Regular expressions to remove numbers from the review
    review_text = re.sub(r'\d+', '',review_text)
    word_list = wordninja.split(review_text)#Works excellent for splitting the words in a sentence.
    word_list = [lemmatizer.lemmatize(w) for w in word_list]
    #Remove all the stop words and include only the words with minimum length of 2.
   # word_list=[spell(w) for w in word_list if w not in stop_words and len(w) > 2]
    modified_text=' '.join([w for w in word_list])
    return modified_text

In [34]:
df['Modified_Cons_Review'] = df.apply(lambda x:text_preprocess(x['Cons']),axis=1)
df['Modified_Pros_Review'] = df.apply(lambda x:text_preprocess(x['Pros']),axis=1)

In [69]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
def bar_plot_topwords(CompanyName,ReviewType):
    common_words = get_top_n_words(df[(df['CompanyName']==CompanyName)][ReviewType], 20)
    df1 = pd.DataFrame(common_words, columns = ['word' , 'count'])
    fig = px.bar(df1, x='word', y='count')
    fig.update_layout(title_text=ReviewType[9:13]+' Review word count top 20 for '+CompanyName, template="plotly_white")
    return fig

In [70]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
def bar_plot_topbigrams(CompanyName,ReviewType):
    common_words = get_top_n_bigram(df[df['CompanyName']==CompanyName][ReviewType], 20)
    df1 = pd.DataFrame(common_words, columns = ['word' , 'count'])
    fig = px.bar(df1, x='word', y='count')
    fig.update_layout(title_text=ReviewType[9:13]+' Review Bi-gram count top 20 for '+CompanyName, template="plotly_white")
    return fig

In [71]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
def bar_plot_toptrigrams(CompanyName,ReviewType):
    common_words = get_top_n_trigram(df[df['CompanyName']==CompanyName][ReviewType], 20)
    df1 = pd.DataFrame(common_words, columns = ['word' , 'count'])
    fig = px.bar(df1, x='word', y='count')
    fig.update_layout(title_text=ReviewType[9:13]+' Review Tri-gram count top 20 for '+CompanyName, template="plotly_white")
    return fig

In [84]:
bar_plot_topwords('Amazon','Modified_Pros_Review')

In [85]:
bar_plot_topbigrams('Amazon','Modified_Pros_Review')

In [86]:
bar_plot_toptrigrams('Amazon','Modified_Pros_Review')

In [87]:
bar_plot_topwords('Amazon','Modified_Cons_Review')

In [88]:
bar_plot_topbigrams('Amazon','Modified_Cons_Review')

In [89]:
bar_plot_toptrigrams('Amazon','Modified_Cons_Review')

In [90]:
bar_plot_toptrigrams('IBM','Modified_Pros_Review')

In [91]:
bar_plot_toptrigrams('IBM','Modified_Cons_Review')

In [92]:
bar_plot_toptrigrams('Microsoft','Modified_Pros_Review')

In [93]:
bar_plot_toptrigrams('Microsoft','Modified_Cons_Review')

In [94]:
bar_plot_toptrigrams('Google','Modified_Pros_Review')

In [95]:
bar_plot_toptrigrams('Google','Modified_Cons_Review')